In [1]:
import numpy as np 
import pandas as pd 
import os
import gc
import random
import time
from PIL import ImageFile, Image

import tensorflow as tf
import efficientnet.tfkeras as efn
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
import tensorflow.keras.models as M
from tensorflow.keras import backend as K
import tensorflow.keras.layers as L
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras import utils
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [2]:
import matplotlib
import sklearn
print("pandas version: ", pd.__version__)
print("numpy version: ", np.__version__)
print("sklearn version: ", sklearn.__version__)
print("matplotlib version: ", matplotlib.__version__)
print("tensorflow version: ", tf.__version__)
print("efficientnet version: 1.1.1")

pandas version:  1.2.2
numpy version:  1.19.5
sklearn version:  0.23.2
matplotlib version:  3.3.2
tensorflow version:  2.4.1
efficientnet version: 1.1.1


###### подробнее про библиотеку efficientnet: https://pypi.org/project/keras-efficientnets/

In [3]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' #gpu
#Whether or not to load truncated image files
ImageFile.LOAD_TRUNCATED_IMAGES = True
#Image size exceeds not limit
Image.MAX_IMAGE_PIXELS = None

In [4]:
seed=42    
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [5]:
# кастомная функция для f1
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
train = pd.read_csv('train_for_final_images_model_2.csv') 

In [7]:
train['image_id']  = train['guid'] + '.jpg'

In [8]:
train.head()

,guid,typology,image_id
0,a1030fb5-af65-4185-9ac4-d0f84f0c268e,прочие,a1030fb5-af65-4185-9ac4-d0f84f0c268e.jpg
1,2e5b0cdf-6668-4e10-a0a1-8b885f6ce61b,прочие,2e5b0cdf-6668-4e10-a0a1-8b885f6ce61b.jpg
2,17dd91e9-a200-432e-8046-278e580f5f3e,прочие,17dd91e9-a200-432e-8046-278e580f5f3e.jpg
3,c17896b3-302c-451a-aaae-18a184988155,прочие,c17896b3-302c-451a-aaae-18a184988155.jpg
4,ce7f88e5-586a-4a99-ac4c-6a8f1820ddc4,прочие,ce7f88e5-586a-4a99-ac4c-6a8f1820ddc4.jpg


In [9]:
train['typology'].value_counts()

фотографии и негативы                                18723
предметы археологии                                  14834
предметы нумизматики                                 14044
предметы прикладного искусства, быта и этнографии    13087
документы                                            12572
предметы печатной продукции                          10208
графика                                               6774
редкие книги                                          3697
предметы естественнонаучной коллекции                 2982
оружие                                                2755
скульптура                                            2422
предметы минералогической коллекции                   2376
предметы техники                                      2374
живопись                                              2016
прочие                                                1888
Name: typology, dtype: int64

In [10]:
train, valid = train_test_split(train, test_size = 0.1, stratify=train['typology'], random_state=seed)

In [11]:
IMG_SIZE = 312
input_shape=(IMG_SIZE, IMG_SIZE, 3)
AUTO = tf.data.experimental.AUTOTUNE
batch_size = 8

In [14]:
datagen_train = ImageDataGenerator(rescale=1./(IMG_SIZE - 1), 
                                   zoom_range=[0.95,1.05],
                                   height_shift_range=0.05,
                                   width_shift_range=0.05,
                                   fill_mode='nearest',
                                   brightness_range=[0.95,1.05]
                                  )

In [15]:
datagen_test = ImageDataGenerator(rescale=1./(IMG_SIZE - 1))

In [16]:
def create_datagen_train(df):
    return datagen_train.flow_from_dataframe(
        dataframe=df,
        directory="images",
        x_col="image_id",
        y_col="typology",
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode='categorical',
        #validation_split=0.15,
        shuffle=True, 
        seed=seed)

In [17]:
def create_datagen_validation(df):
    return datagen_test.flow_from_dataframe(
        dataframe=df,
        directory="images",
        x_col="image_id",
        y_col="typology",
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode='categorical',
        #validation_split=0.15,
        shuffle=False,
        seed=seed)

Загружу общедоступные предобученные веса на Imagenet на архитектуре EfficientNetB6, на основе этой архитерктуры достроим нейронную сеть подробнее про архитектуру https://paperswithcode.com/paper/fixing-the-train-test-resolution-discrepancy-2

Сохраню веса (для случая недоступности интернета)

In [18]:
#base_model = efn.EfficientNetB6(weights="imagenet", include_top=False, input_shape=input_shape)
#base_model.save("EfficientNetB6.h5")

In [19]:
base_model = efn.EfficientNetB6(weights="EfficientNetB6.h5", include_top=False, input_shape=input_shape)
base_model.trainable = True

In [20]:
def architecture_NN():
    inp = L.Input(shape=input_shape)
    x = base_model(inp)
    x = L.GlobalAveragePooling2D()(x)
    out = L.Dense(15, activation = 'softmax', kernel_initializer='glorot_uniform')(x)
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    return model

In [21]:
architecture_NN().summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 312, 312, 3)]     0         
_________________________________________________________________
efficientnet-b6 (Functional) (None, 10, 10, 2304)      40960136  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 15)                34575     
Total params: 40,994,711
Trainable params: 40,770,279
Non-trainable params: 224,432
_________________________________________________________________


In [22]:
LR = 0.00009
num_epochs = 6
checkpoint = ModelCheckpoint(filepath='Models_images/long_models_for_inference/best_model.5', monitor=['val_loss'], verbose=1, mode='min')
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1, mode='min')
train_generator = create_datagen_train(train)
valid_generator = create_datagen_validation(valid)
model = architecture_NN()
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(LR), metrics=[f1_m])

Found 99676 validated image filenames belonging to 15 classes.
Found 11076 validated image filenames belonging to 15 classes.


In [23]:
start_time = time.time()
history = model.fit(train_generator, 
                    validation_data=valid_generator,
                    callbacks=[checkpoint, es],
                    epochs = num_epochs)
model.save(f"Models_images/long_models_for_inference/model_final_predict_image_.h5")
print(f"hours: {(time.time()-start_time)/3600}")

Epoch 1/6
 1830/12460 [===>..........................] - ETA: 1:34:48 - loss: 1.2656 - f1_m: 0.5153

/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/JpegImagePlugin.py:791: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


12460/12460 [==============================] - 7170s 574ms/step - loss: 0.7812 - f1_m: 0.7199 - val_loss: 0.3798 - val_f1_m: 0.8684

Epoch 00001: saving model to Models_images/long_models_for_inference/best_model
INFO:tensorflow:Assets written to: Models_images/long_models_for_inference/best_model/assets
Epoch 2/6
 3219/12460 [======>.......................] - ETA: 1:24:42 - loss: 0.3976 - f1_m: 0.8647

/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/JpegImagePlugin.py:791: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


12460/12460 [==============================] - 7096s 570ms/step - loss: 0.3925 - f1_m: 0.8651 - val_loss: 0.3166 - val_f1_m: 0.8935

Epoch 00002: saving model to Models_images/long_models_for_inference/best_model
INFO:tensorflow:Assets written to: Models_images/long_models_for_inference/best_model/assets
Epoch 3/6
  148/12460 [..............................] - ETA: 2:04:09 - loss: 0.2876 - f1_m: 0.8899

...

12460/12460 [==============================] - 7120s 571ms/step - loss: 0.3081 - f1_m: 0.8910 - val_loss: 0.3316 - val_f1_m: 0.8976

Epoch 00003: saving model to Models_images/long_models_for_inference/best_model
INFO:tensorflow:Assets written to: Models_images/long_models_for_inference/best_model/assets
Epoch 4/6
 2445/12460 [====>.........................] - ETA: 1:31:43 - loss: 0.2588 - f1_m: 0.9070

/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/JpegImagePlugin.py:791: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


12460/12460 [==============================] - 7056s 566ms/step - loss: 0.2595 - f1_m: 0.9069 - val_loss: 0.3204 - val_f1_m: 0.8982

Epoch 00004: saving model to Models_images/long_models_for_inference/best_model
INFO:tensorflow:Assets written to: Models_images/long_models_for_inference/best_model/assets
Epoch 5/6
  320/12460 [..............................] - ETA: 1:47:10 - loss: 0.1902 - f1_m: 0.9269

/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/JpegImagePlugin.py:791: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


12460/12460 [==============================] - 6756s 542ms/step - loss: 0.2211 - f1_m: 0.9192 - val_loss: 0.3201 - val_f1_m: 0.9071

Epoch 00005: saving model to Models_images/long_models_for_inference/best_model
INFO:tensorflow:Assets written to: Models_images/long_models_for_inference/best_model/assets
Restoring model weights from the end of the best epoch.
Epoch 00005: early stopping
hours: 9.849952221976386


In [26]:
len(history.model.weights)

852

In [27]:
model.save(f"Models_images/long_models_for_inference/model_images.h5")

In [28]:
predict = model.predict(valid_generator)

/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/JpegImagePlugin.py:791: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


In [29]:
import pickle
with open('Models_text/LabelEncoder.pickle', 'rb') as handle:
    le = pickle.load(handle)

In [30]:
valid['typology_predict'] = le.inverse_transform(np.argmax(predict, axis=1))

In [31]:
from sklearn.metrics import f1_score
f1_score(valid['typology'], valid['typology_predict'], average='macro')

0.8618282719560516

In [32]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(valid['typology'], valid['typology_predict'])

In [33]:
cm_df = pd.DataFrame(cm,
                     index = le.classes_.tolist(), 
                     columns = le.classes_.tolist())

In [34]:
cm_df

,графика,документы,живопись,оружие,предметы археологии,предметы естественнонаучной коллекции,предметы минералогической коллекции,предметы нумизматики,предметы печатной продукции,"предметы прикладного искусства, быта и этнографии",предметы техники,прочие,редкие книги,скульптура,фотографии и негативы
графика,552,0,21,0,0,1,0,0,82,1,0,0,0,0,20
документы,1,1131,0,0,0,0,0,0,111,0,0,5,9,0,0
живопись,22,0,171,0,0,0,0,0,5,3,0,0,0,0,1
оружие,0,0,0,271,0,0,0,0,0,4,1,0,0,0,0
предметы археологии,0,0,0,5,1389,16,20,1,0,51,0,0,0,2,0
предметы естественнонаучной коллекции,0,0,0,1,23,250,12,2,0,9,1,0,0,0,0
предметы минералогической коллекции,0,0,0,0,14,0,222,0,0,0,0,2,0,0,0
предметы нумизматики,4,0,0,0,2,3,0,1389,1,5,0,0,0,0,0
предметы печатной продукции,19,108,0,0,0,0,0,4,726,0,0,3,104,0,57
"предметы прикладного искусства, быта и этнографии",2,0,2,9,18,5,0,31,0,1164,32,21,0,25,0


In [35]:
import sklearn.metrics
for i in valid['typology'].unique():
    print(i)
    print(sklearn.metrics.accuracy_score(valid.query(f"typology=='{i}'")['typology'], valid.query(f"typology=='{i}'")['typology_predict']))

предметы печатной продукции
0.7110675808031341
предметы техники
0.7848101265822784
редкие книги
0.7783783783783784
прочие
0.5502645502645502
фотографии и негативы
0.9754273504273504
оружие
0.9818840579710145
документы
0.8997613365155132
предметы археологии
0.9359838274932615
предметы нумизматики
0.9893162393162394
предметы прикладного искусства, быта и этнографии
0.8892284186401833
графика
0.8153618906942393
скульптура
1.0
предметы естественнонаучной коллекции
0.8389261744966443
предметы минералогической коллекции
0.9327731092436975
живопись
0.8465346534653465


In [39]:
m = tf.keras.models.load_model(f"Models_images/long_models_for_inference/best_model/", custom_objects={'f1_m': f1_m})

In [50]:
predict = m.predict(valid_generator)

/home/eugene/anaconda3/lib/python3.8/site-packages/PIL/JpegImagePlugin.py:791: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


In [51]:
valid['typology_predict'] = le.inverse_transform(np.argmax(predict, axis=1))

In [52]:
f1_score(valid['typology'], valid['typology_predict'], average='macro')

0.8860664226623326

In [53]:
cm = confusion_matrix(valid['typology'], valid['typology_predict'])

In [54]:
cm_df = pd.DataFrame(cm,
                     index = le.classes_.tolist(), 
                     columns = le.classes_.tolist())

In [55]:
cm_df

,графика,документы,живопись,оружие,предметы археологии,предметы естественнонаучной коллекции,предметы минералогической коллекции,предметы нумизматики,предметы печатной продукции,"предметы прикладного искусства, быта и этнографии",предметы техники,прочие,редкие книги,скульптура,фотографии и негативы
графика,574,0,44,0,1,0,2,0,42,4,0,1,0,0,9
документы,1,1168,0,0,0,0,0,0,76,0,0,5,6,0,1
живопись,16,0,178,0,0,0,0,0,4,2,0,1,0,1,0
оружие,0,0,0,275,0,0,0,0,0,1,0,0,0,0,0
предметы археологии,0,0,0,3,1415,8,14,3,2,38,0,1,0,0,0
предметы естественнонаучной коллекции,0,0,0,2,21,244,22,3,0,6,0,0,0,0,0
предметы минералогической коллекции,0,0,0,0,4,0,234,0,0,0,0,0,0,0,0
предметы нумизматики,2,0,0,1,0,0,0,1393,0,5,1,1,0,0,1
предметы печатной продукции,39,140,1,0,0,0,0,1,704,0,0,3,92,0,41
"предметы прикладного искусства, быта и этнографии",0,0,1,17,32,3,1,25,0,1161,36,21,2,10,0


In [46]:
import sklearn.metrics
for i in valid['typology'].unique():
    print(i)
    print(sklearn.metrics.accuracy_score(valid.query(f"typology=='{i}'")['typology'], valid.query(f"typology=='{i}'")['typology_predict']))

предметы печатной продукции
0.6895200783545543
предметы техники
0.9282700421940928
редкие книги
0.7972972972972973
прочие
0.8095238095238095
фотографии и негативы
0.9556623931623932
оружие
0.9963768115942029
документы
0.9291964996022275
предметы археологии
0.9535040431266847
предметы нумизматики
0.9921652421652422
предметы прикладного искусства, быта и этнографии
0.8869365928189458
графика
0.8478581979320532
скульптура
1.0
предметы естественнонаучной коллекции
0.8187919463087249
предметы минералогической коллекции
0.9831932773109243
живопись
0.8811881188118812


In [47]:
m.save(f"Models_images/long_models_for_inference/model_images_2.h5")

In [49]:
m = tf.keras.models.load_model(f"Models_images/long_models_for_inference/model_images_2.h5", custom_objects={'f1_m': f1_m})